## Transformacja wyników badań laboratoryjnych, z pliku txt do tableki excel

Wczytanie Bibliotek

In [1]:
import pandas as pd
import re

Import danych

In [2]:
# Wczytanie pliku tekstowego do Pandas DataFrame
txt_file_path = 'C:/dane.txt'
df = pd.read_csv(txt_file_path, sep="/n", header=None)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df

pattern = 'godz'

slownik = {}
aktualny_klucz = None

for wiersz in df[0]:
    if pattern in wiersz:
        aktualny_klucz = wiersz
        slownik[aktualny_klucz] = []
        print(aktualny_klucz)
    elif aktualny_klucz is not None:
        slownik[aktualny_klucz].append(wiersz)

pd.set_option('display.max_colwidth', None)
df = pd.DataFrame.from_dict(slownik, orient='index').transpose()
df

23-02-2024 godz. 13:47, 4936167521
23-02-2024 godz. 13:45, 1700517131
23-02-2024 godz. 13:25, 7610485431
21-02-2024 godz. 13:55, 4937167523


C:\Users\Szymon\AppData\Local\Temp\ipykernel_11680\3048199027.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(txt_file_path, sep="/n", header=None)


,"23-02-2024 godz. 13:47, 4936167521","23-02-2024 godz. 13:45, 1700517131","23-02-2024 godz. 13:25, 7610485431","21-02-2024 godz. 13:55, 4937167523"
0,HBSAG: Antygen HBs niereaktywny,HCV: Przeciwciała anty HCV niereaktywny,"KOR: Kortyzol 9,74 µg/dl","MOCZ: Przejrzystość przejrzys , Barwa zolta , Ciężar właściwy 1,020 g/ml, pH 6,0 , Glukoza Nieobecna mg/dl, Ketony Nieobecne , Urobilinogen Nieobecny µmol/l, Bilirubina Nieobecna mg/dl, Białko Nieobecne mg/dl, Azotyny Nieobecne , Erytrocyty Nieobecna , Leukocyty Nieobecna"
1,ALT: 15 U/l,"KFOL: Kwas foliowy 7,32 ng/ml",None,"MBO: Nabłonki płaskie nieliczne wpw, Leukocyty 0-1 wpw, Bakterie nieliczne w prep."
2,"TSH: 1,260 µIU/ml","B12: Witamina B12 462,00 pg/ml",None,None
3,"LIPID: Cholesterol całkowity 181 mg/dl, Cholesterol HDL 67,9 mg/dl, Cholesterol nie-HDL 113,10 mg/dl, Cholesterol LDL 102,10 mg/dl, Triglicerydy 55 mg/dl",None,None,None
4,"CRPIL: CRP 0,21 mg/l",None,None,None
5,"FT4: 1,15 ng/dl",None,None,None
6,"FT3: 2,89 pg/ml",None,None,None
7,LIP: Lipaza 28 U/l,None,None,None
8,ASOIL: ASO 6 IU/ml,None,None,None
9,"FE: Żelazo 10,6 µg/dl",None,None,None


Wykrywanie badań pierwszego typu

In [3]:
# Funkcja do przetwarzania jednej komórki i wyciągania danych
def extract_data(cell):
    if isinstance(cell, str):
        #Wzorzec do wykrywania danych w formacie "Ticker: Nazwa Badania Wynik Jednostka"
        pattern_full = r'^(\w+):\s*([\w\s\(\)]+)\s([\w\s\.,<>-]+)\s([\w/µ%]+)$'
        match_full = re.match(pattern_full, cell)
        if match_full:
            ticker = match_full.group(1)
            nazwa_badania = match_full.group(2)
            wynik = match_full.group(3)
            jednostka = match_full.group(4)
            return pd.Series([ticker, nazwa_badania, wynik, jednostka])
        
        # Wzorzec do wykrywania danych w formacie "Ticker: Wynik Jednostka"
        pattern_simple = r'^(\w+):\s*([\w\s\.,<>-]+)\s([\w/µ%]+)$'
        match_simple = re.match(pattern_simple, cell)
        if match_simple:
            ticker = match_simple.group(1)
            wynik = match_simple.group(2)
            jednostka = match_simple.group(3)
            return pd.Series([ticker, ticker, wynik, jednostka])

      

    return pd.Series([None, None, None, None])

# Funkcja do wyciągania daty, godziny i ID z nazwy kolumny
def extract_column_info(col_name):
    pattern = r'(\d{2}-\d{2}-\d{4}) godz\. (\d{2}:\d{2}), (\d{10})'
    match = re.match(pattern, col_name)
    if match:
        data = match.group(1)
        godzina = match.group(2)
        id_ = match.group(3)
        return data, godzina, id_
    return None, None, None

# Inicjalizacja df1
df1 = pd.DataFrame(columns=["Data", "Godzina", "ID", "Ticker", "Nazwa Badania", "Wynik", "Jednostka"])

# Przetwarzanie danych
for col_name in df.columns:
    # Wyciągnięcie daty, godziny i ID z nazwy kolumny
    data, godzina, id_ = extract_column_info(col_name)
    
    # Zastosowanie funkcji extract_data do każdej komórki w kolumnie
    temp_df = df[col_name].apply(extract_data)
    temp_df = temp_df.dropna()  # Usunięcie wierszy, które nie pasowały do wzorca
    
    # Przypisanie wyciągniętej daty, godziny i ID do nowych kolumn
    temp_df['Data'] = data
    temp_df['Godzina'] = godzina
    temp_df['ID'] = id_
    
    # Uporządkowanie kolumn
    temp_df = temp_df[["Data", "Godzina", "ID", 0, 1, 2, 3]]
    temp_df.columns = ["Data", "Godzina", "ID", "Ticker", "Nazwa Badania", "Wynik", "Jednostka"]
    
    # Dodanie wyników do df1
    df1 = pd.concat([df1, temp_df], ignore_index=True)

# Zakładając, że df1 jest już zdefiniowane

# Przetwarzanie wierszy dla tickerów HBSAG oraz HCV
mask = df1['Ticker'].isin(['HBSAG', 'HCV'])

# Dla tych wierszy połącz wartość z kolumny "Nazwa Badania" z wartością z kolumny "Wynik"
df1.loc[mask, 'Nazwa Badania'] = df1.loc[mask, 'Nazwa Badania'] + ' ' + df1.loc[mask, 'Wynik']

# Zastąp wartość w kolumnie "Wynik" wartością z kolumny "Jednostka"
df1.loc[mask, 'Wynik'] = df1.loc[mask, 'Jednostka']

# W kolumnie "Jednostka" wstaw wartość None
df1.loc[mask, 'Jednostka'] = None

df1

,Data,Godzina,ID,Ticker,Nazwa Badania,Wynik,Jednostka
0,23-02-2024,13:47,4936167521,HBSAG,Antygen HBs,niereaktywny,None
1,23-02-2024,13:47,4936167521,ALT,ALT,15,U/l
2,23-02-2024,13:47,4936167521,TSH,TSH,"1,260",µIU/ml
3,23-02-2024,13:47,4936167521,CRPIL,CRP,"0,21",mg/l
4,23-02-2024,13:47,4936167521,FT4,FT4,"1,15",ng/dl
5,23-02-2024,13:47,4936167521,FT3,FT3,"2,89",pg/ml
6,23-02-2024,13:47,4936167521,LIP,Lipaza,28,U/l
7,23-02-2024,13:47,4936167521,ASOIL,ASO,6,IU/ml
8,23-02-2024,13:47,4936167521,FE,Żelazo,"10,6",µg/dl
9,23-02-2024,13:47,4936167521,UA,Kwas moczowy,"3,4",mg/dl


Wykrywanie badań drugiego typu

In [5]:

# Funkcja do przetwarzania jednej komórki i wyciągania danych - Skrypt 2
def extract_data_skrypt2(cell):
    if isinstance(cell, str):
        # Specjalne traktowanie tickeru "MOCZ"
        if cell.startswith("MOCZ:"):
            ticker = "MOCZ"
            details = cell[len("MOCZ:"):].strip()
            pattern_mocz = r'([\w\s\-\/\(\)]+?)\s([\w\.,\-\/\(\)]+)\s?(µmol/l|mg/dl|g/ml|None)?'
            results = re.findall(pattern_mocz, details)
            
            rows = []
            for result in results:
                nazwa_badania = result[0].strip()
                wynik = result[1].strip().replace(",", "")
                jednostka = result[2].strip() if result[2] else None
                rows.append([ticker, nazwa_badania, wynik, jednostka])
            
            return pd.DataFrame(rows, columns=["Ticker", "Nazwa Badania", "Wynik", "Jednostka"])
        
        # Wzorzec do wykrywania danych dla pozostałych tickerów
        pattern_full = r'^(\w+):\s*(.+)$'
        match_full = re.match(pattern_full, cell)
        if match_full:
            ticker = match_full.group(1)
            details = match_full.group(2).strip()
            results = re.findall(r'([\w\s\-\/\(\)]+?)\s([<>]?\s?[\d,\.]+|----)\s([\w/µ%]+)', details)
            
            rows = []
            for result in results:
                nazwa_badania = result[0].strip()
                wynik = result[1].strip()
                jednostka = result[2]
                rows.append([ticker, nazwa_badania, wynik, jednostka])
            
            return pd.DataFrame(rows, columns=["Ticker", "Nazwa Badania", "Wynik", "Jednostka"])
    
    return pd.DataFrame(columns=["Ticker", "Nazwa Badania", "Wynik", "Jednostka"])

# Funkcja do wyciągania daty, godziny i ID z nazwy kolumny - Skrypt 2
def extract_column_info_skrypt2(col_name):
    pattern = r'(\d{2}-\d{2}-\d{4}) godz\. (\d{2}:\d{2}), (\d{10})'
    match = re.match(pattern, col_name)
    if match:
        data = match.group(1)
        godzina = match.group(2)
        id_ = match.group(3)
        return data, godzina, id_
    return None, None, None

# Inicjalizacja df2 - Skrypt 2
df2 = pd.DataFrame(columns=["Data", "Godzina", "ID", "Ticker", "Nazwa Badania", "Wynik", "Jednostka"])

# Przetwarzanie danych - Skrypt 2
przetworzone_wiersze_skrypt2 = set()

for col_name in df.columns:
    data, godzina, id_ = extract_column_info_skrypt2(col_name)
    
    temp_df = df[col_name].apply(extract_data_skrypt2)
    temp_df = pd.concat(temp_df.values).reset_index(drop=True)
    
    temp_df['Data'] = data
    temp_df['Godzina'] = godzina
    temp_df['ID'] = id_
    
    temp_df = temp_df[["Data", "Godzina", "ID", "Ticker", "Nazwa Badania", "Wynik", "Jednostka"]]
    
    for _, row in temp_df.iterrows():
        row_tuple = tuple(row)
        if row_tuple not in przetworzone_wiersze_skrypt2:
            przetworzone_wiersze_skrypt2.add(row_tuple)
            df2 = pd.concat([df2, pd.DataFrame([row], columns=df2.columns)], ignore_index=True)

#--------------------Usunięcie błędznie zaczytanego Ciężar właściwy------------------

# Znajdowanie indeksów wierszy, które zawierają wartość "Ciężar" w kolumnie "Nazwa Badania"
index_to_remove = df2[df2['Nazwa Badania'].str.contains("Ciężar", na=False)].index

# Usunięcie znalezionych wierszy oraz wiersza poniżej
if len(index_to_remove) > 0:
    indices_to_drop = index_to_remove.tolist() + (index_to_remove + 1).tolist()
    indices_to_drop = list(set(indices_to_drop))  # Usunięcie ewentualnych duplikatów
    df2 = df2.drop(indices_to_drop).reset_index(drop=True)

#-------------------Poprawne zaczytanie ciężar właściwy---------------------------------------

# Funkcja do wyciągania daty, godziny i ID z nazwy kolumny
def extract_column_info_skrypt2(col_name):
    pattern = r'(\d{2}-\d{2}-\d{4}) godz\. (\d{2}:\d{2}), (\d{10})'
    match = re.match(pattern, col_name)
    if match:
        data = match.group(1)
        godzina = match.group(2)
        id_ = match.group(3)
        return data, godzina, id_
    return None, None, None

# Inicjalizacja df3
df3 = pd.DataFrame(columns=["Data", "Godzina", "ID", "Ticker", "Nazwa Badania", "Wynik", "Jednostka"])

# Wzorzec do wykrywania struktury "Ciężar właściwy Wynik Jednostka"
pattern_ciezar = r"Ciężar właściwy\s([\d,\.]+)\s([\w/µ%]+)"

# Przetwarzanie danych
for col_name in df.columns:
    data, godzina, id_ = extract_column_info_skrypt2(col_name)
    
    for index, cell in df[col_name].items():
        if isinstance(cell, str):
            match = re.search(pattern_ciezar, cell)
            if match:
                ticker = "MOCZ"
                nazwa_badania = "Ciężar właściwy"
                wynik = match.group(1)
                jednostka = match.group(2)
                
                # Dodanie wiersza do df3
                new_row = pd.DataFrame({
                    "Data": [data],
                    "Godzina": [godzina],
                    "ID": [id_],
                    "Ticker": [ticker],
                    "Nazwa Badania": [nazwa_badania],
                    "Wynik": [wynik],
                    "Jednostka": [jednostka]
                })
                
                df3 = pd.concat([df3, new_row], ignore_index=True)
                
#---------------------Wyszukiwanie MBO---------------------------------------
# Funkcja do wyciągania daty, godziny i ID z nazwy kolumny
def extract_column_info_skrypt2(col_name):
    pattern = r'(\d{2}-\d{2}-\d{4}) godz\. (\d{2}:\d{2}), (\d{10})'
    match = re.match(pattern, col_name)
    if match:
        data = match.group(1)
        godzina = match.group(2)
        id_ = match.group(3)
        return data, godzina, id_
    return None, None, None

# Inicjalizacja df4
df4 = pd.DataFrame(columns=["Data", "Godzina", "ID", "Ticker", "Nazwa Badania", "Wynik", "Jednostka"])

# Wzorzec do wykrywania struktury "MBO: Nabłonki płaskie nieliczne wpw, Leukocyty 0-1 wpw, Bakterie nieliczne w prep."
pattern_mbo = r"MBO:\s*Nabłonki płaskie\s([\w\s\.,-]+)\s(wpw),\s*Leukocyty\s([\w\s\.,-]+)\s(wpw),\s*Bakterie\s([\w\s\.,-]+)\s(w prep\.)"

# Przetwarzanie danych
for col_name in df.columns:
    data, godzina, id_ = extract_column_info_skrypt2(col_name)
    
    for index, cell in df[col_name].items():
        if isinstance(cell, str):
            match = re.search(pattern_mbo, cell)
            if match:
                ticker = "MBO"
                
                # Nazwy badań
                nazwa_badania1 = "Nabłonki płaskie"
                nazwa_badania2 = "Leukocyty"
                nazwa_badania3 = "Bakterie"
                
                # Wyniki badań
                wynik1 = match.group(1).strip()
                jednostka1 = match.group(2).strip()
                
                wynik2 = match.group(3).strip()
                jednostka2 = match.group(4).strip()
                
                wynik3 = match.group(5).strip()
                jednostka3 = match.group(6).strip()
                
                # Tworzenie nowych wierszy do df4
                rows_to_add = [
                    {"Data": data, "Godzina": godzina, "ID": id_, "Ticker": ticker, "Nazwa Badania": nazwa_badania1, "Wynik": wynik1, "Jednostka": jednostka1},
                    {"Data": data, "Godzina": godzina, "ID": id_, "Ticker": ticker, "Nazwa Badania": nazwa_badania2, "Wynik": wynik2, "Jednostka": jednostka2},
                    {"Data": data, "Godzina": godzina, "ID": id_, "Ticker": ticker, "Nazwa Badania": nazwa_badania3, "Wynik": wynik3, "Jednostka": jednostka3},
                ]
                
                df4 = pd.concat([df4, pd.DataFrame(rows_to_add)], ignore_index=True)

In [6]:
df2

,Data,Godzina,ID,Ticker,Nazwa Badania,Wynik,Jednostka
0,23-02-2024,13:47,4936167521,LIPID,Cholesterol całkowity,181,mg/dl
1,23-02-2024,13:47,4936167521,LIPID,Cholesterol HDL,"67,9",mg/dl
2,23-02-2024,13:47,4936167521,LIPID,Cholesterol nie-HDL,"113,10",mg/dl
3,23-02-2024,13:47,4936167521,LIPID,Cholesterol LDL,"102,10",mg/dl
4,23-02-2024,13:47,4936167521,LIPID,Triglicerydy,55,mg/dl
5,23-02-2024,13:47,4936167521,CRPIL,CRP,"0,21",mg/l
6,23-02-2024,13:47,4936167521,LIP,Lipaza,28,U/l
7,23-02-2024,13:47,4936167521,ASOIL,ASO,6,IU/ml
8,23-02-2024,13:47,4936167521,FE,Żelazo,"10,6",µg/dl
9,23-02-2024,13:47,4936167521,UA,Kwas moczowy,"3,4",mg/dl


In [7]:
df3

,Data,Godzina,ID,Ticker,Nazwa Badania,Wynik,Jednostka
0,21-02-2024,13:55,4937167523,MOCZ,Ciężar właściwy,"1,020",g/ml


In [8]:
df4

,Data,Godzina,ID,Ticker,Nazwa Badania,Wynik,Jednostka
0,21-02-2024,13:55,4937167523,MBO,Nabłonki płaskie,nieliczne,wpw
1,21-02-2024,13:55,4937167523,MBO,Leukocyty,0-1,wpw
2,21-02-2024,13:55,4937167523,MBO,Bakterie,nieliczne,w prep.


Łączenie tabel

In [6]:
# Łączenie wyników obu skryptów
df_combined = pd.concat([df1, df2, df3, df4], axis=0)
df_sorted = df_combined.sort_values(by=['Data', 'Godzina'], ascending=[False, False])
df_unique = df_sorted.drop_duplicates()
df_combined_reset = df_unique.reset_index(drop=True)
df_combined_reset


,Data,Godzina,ID,Ticker,Nazwa Badania,Wynik,Jednostka
0,23-02-2024,13:47,4936167521,HBSAG,Antygen HBs,niereaktywny,None
1,23-02-2024,13:47,4936167521,ALT,ALT,15,U/l
2,23-02-2024,13:47,4936167521,TSH,TSH,"1,260",µIU/ml
3,23-02-2024,13:47,4936167521,CRPIL,CRP,"0,21",mg/l
4,23-02-2024,13:47,4936167521,FT4,FT4,"1,15",ng/dl
5,23-02-2024,13:47,4936167521,FT3,FT3,"2,89",pg/ml
6,23-02-2024,13:47,4936167521,LIP,Lipaza,28,U/l
7,23-02-2024,13:47,4936167521,ASOIL,ASO,6,IU/ml
8,23-02-2024,13:47,4936167521,FE,Żelazo,"10,6",µg/dl
9,23-02-2024,13:47,4936167521,UA,Kwas moczowy,"3,4",mg/dl


Eksport DataFrame do pliku csv

In [9]:
import pandas as pd

# Funkcja eksportująca DataFrame do pliku Excel z możliwością określenia ścieżki
def eksportuj_do_excela(df, sciezka):
    # Sprawdzenie, czy ścieżka ma poprawne rozszerzenie
    if not sciezka.endswith('.xlsx'):
        raise ValueError("Ścieżka pliku musi kończyć się na '.xlsx'")
    
    try:
        df.to_excel(sciezka, index=False)
        print(f"Plik został pomyślnie zapisany w lokalizacji: {sciezka}")
    except Exception as e:
        print(f"Wystąpił błąd podczas zapisywania pliku: {e}")

# Przykładowe użycie funkcji
sciezka_do_zapisu = 'D:/df_combined_reset.xlsx'  # Zmień na swoją ścieżkę z rozszerzeniem .xlsx
eksportuj_do_excela(df_combined_reset, sciezka_do_zapisu)


Plik został pomyślnie zapisany w lokalizacji: D:/df_combined_reset.xlsx
